## Import

In [1]:
from pypfopt.expected_returns import mean_historical_return, ema_historical_return, capm_return, returns_from_prices
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.cla import CLA
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models, exceptions
from pypfopt import EfficientFrontier, CLA
from mdutils.mdutils import MdUtils
from mdutils.tools.Table import Table
from tkinter import * # __all__
from tkinter import filedialog
from time import gmtime, strftime, localtime
from copy import deepcopy
from tqdm import tqdm
from datetime import timedelta, datetime
from collections import OrderedDict
from pyfolio.utils import APPROX_BDAYS_PER_MONTH, MM_DISPLAY_UNIT
from pypfopt.cla import CLA

import tkinter.ttk as ttk
import tkinter.messagebox as msgbox
import matplotlib.pyplot as plt
import xlwings as xw
import pandas as pd
import numpy as np
import FinanceDataReader as fdr
import pandas as pd
import numpy as np
import pyfolio as pf
import seaborn as sns
import pypfopt
import empyrical
import markdown
import os, re, copy, warnings

warnings.filterwarnings(action='ignore')
plt.style.use("seaborn-deep")
plt.rcParams['font.family']= "gulim"

D:\python2\python-3.8.9.amd64\lib\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


WARN: Could not locate executable g77
WARN: Could not locate executable f77
WARN: Could not locate executable ifort
WARN: Could not locate executable ifl
WARN: Could not locate executable f90
WARN: Could not locate executable DF
WARN: Could not locate executable efl
WARN: Could not locate executable gfortran
WARN: Could not locate executable f95
WARN: Could not locate executable g95
WARN: Could not locate executable efort
WARN: Could not locate executable efc
WARN: Could not locate executable flang
WARN: don't know how to compile Fortran code on platform 'nt'


WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


## Functions

* about data crwaling

In [2]:
def read_xlsx(name):
    instance = xw.App(visible = False)
    xlsx_data = xw.Book(name).sheets[0]
    df = xlsx_data.range('A1').options(pd.DataFrame, index = False, expand = 'table').value
    instance.quit()
    instance.kill()
    return df

def read_data(name):
    try :
        output = pd.read_excel(name)
    except ValueError:    
        output = read_xlsx(name)
    return output

def get_stock_list(data_path):
    if os.path.isfile(data_path + "df_krx.xlsx") & os.path.isfile(data_path + "df_america.xlsx"):
        df_krx = read_data(data_path + "df_krx.xlsx")
        df_america = read_data(data_path + "df_america.xlsx")
        
    else:
        df_krx = fdr.StockListing('KRX')
        df_sp500 = fdr.StockListing('S&P500')
        df_nas = fdr.StockListing('NASDAQ')
        df_nyse = fdr.StockListing('NYSE')
        df_amex = fdr.StockListing('AMEX')

        df_america = pd.concat([df_nas, df_nyse, df_amex]).drop_duplicates()
        append_code = [x for x in df_sp500.Symbol if x not in list(df_america.Symbol)]
        append_df = df_sp500.loc[df_sp500.Symbol.isin(append_code)]
        del append_df['Sector']
        df_america = pd.concat([df_america, append_df])
        df_america.index = [x for x in range(df_america.shape[0])]
        
        df_krx.to_excel(data_path + "df_krx.xlsx", index = False)
        df_america.to_excel(data_path + "df_america.xlsx", index = False)
    
    return df_krx, df_america

def change_name(symbol, df, options):
    if options:
        output = df.loc[df.Symbol == symbol, 'Name'].values[0]
    else:
        output = df.loc[df['상품번호'] == symbol, '종목명'].values[0]
    return output

def make_df(name, df):
    temp_df = pd.DataFrame(df.Close)
    temp_df.columns = ['{}'.format(name)]
    return temp_df.reset_index()

def get_currency(data_path):
    # 환율 가져오기
    if os.path.isfile(data_path + "currency.xlsx"):
        currency = read_data(data_path + "currency.xlsx")

    start_date = np.max(currency.Date)

    try :
        append_curr = fdr.DataReader("USD/KRW", start = start_date)
        append_curr = append_curr.reset_index()

        currency = currency.loc[currency.Date < start_date, :]
        currency = pd.concat([currency, append_curr])
        currency.index = [x for x in range(currency.shape[0])]
        currency.to_excel(data_path + "currency.xlsx", index = False)

    except:
        print("추가행 없음")
        pass
    return currency, start_date

def load_data(data_path):
    # data 불러오기
    if os.path.isfile(data_path + "df_infos.xlsx"):
        df_infos = read_data(data_path + "df_infos.xlsx")
        df_krx, df_america = get_stock_list(data_path)
        df_america = df_america.loc[:, ['Symbol', 'Name']]
        df_krx = df_krx.loc[:, ['Symbol', 'Name']]
        
    else:
        df_krx, df_america = get_stock_list(data_path)
        df_america = df_america.loc[:, ['Symbol', 'Name']]
        df_krx = df_krx.loc[:, ['Symbol', 'Name']]
        df_infos = pd.concat([df_krx, df_america])
        df_infos = df_infos.dropna()
        df_infos.index = [x for x in range(df_infos.shape[0])]

        symbol_list = []

        for symbol, name in zip(df_infos.Symbol, df_infos.Name):
            if " PR " in symbol:
                symbol_list.append(re.sub(" PR ","/",symbol))
            elif (".U" in symbol) and ("Units" in name):
                symbol_list.append(re.sub(".U","/UN",symbol))
            elif "\." in symbol:
                symbol_list.append(re.sub("\.", "/", symbol))
            elif " RT" in symbol:
                symbol_list.append(symbol.split(" ")[0])
            elif symbol == 'BRKB':
                symbol_list.append("BRK/B")
            elif symbol == 'BFB':
                symbol_list.append("BF/B")
            else:
                symbol_list.append(symbol)

        df_infos['WINK_symbols'] = symbol_list
        df_infos.to_excel(data_path + "df_infos.xlsx", index = False)
    return df_infos

def get_include_data(data, df_infos, currency, data_path, min_date):
    if '계좌번호' in list(data.columns):
        cate_list = ['주식', 'NYSE', 'AMEX', 'NASD']

        data = data.loc[data['상품구분'].isin(cate_list)]
        account = [True if "-01" in x else False for x in data['계좌번호']]
        data = data.loc[account, :]

        # 국내 주식
        stock_list = data.loc[data['상품구분'] == '주식']['종목명']
        symbol_list = list(df_infos.loc[df_infos.Name.isin(stock_list)].Symbol)
        th = len(symbol_list)

        # 미국 주식
        stock_list = data.loc[data['상품구분'] != '주식']['상품번호']
        symbol_list.extend(list(df_infos.loc[df_infos.Symbol.isin(stock_list)].Symbol))

        start_idx = 0

        for idx, symbol in enumerate(tqdm(symbol_list[start_idx:])):
            if " " in symbol:
                search_symbol = symbol
                symbol = df_infos.loc[df_infos.Symbol == symbol, :].WINK_symbols.values[0]
            else:
                search_symbol = symbol

            try:
                df = fdr.DataReader(symbol, start = min_date)

            except ValueError:
                start_idx += 1
                continue

            if df.shape[0] == 0:
                start_idx += 1
                continue

            if start_idx >= th:
                name = change_name(search_symbol, data, False)
            else:
                name = change_name(search_symbol, df_infos, True)

            if start_idx == 0 :
                temp_stock_df = make_df(name, df)
            else:
                temp_df = make_df(name, df)
                temp_stock_df = pd.merge(temp_stock_df, temp_df, on ='Date', how = 'outer')

            start_idx += 1

        temp_stock_df = temp_stock_df.sort_values("Date")
        temp_stock_df.index = [x for x in range(temp_stock_df.shape[0])]

        # 한/미 증시 시차로 인한 NA 값 발생
        if temp_stock_df.iloc[-1:].isna().sum().sum() > 0:
            temp_stock_df = temp_stock_df.iloc[:-1,:]

        # 미국 종목 환율 반영
        select_col = [0]
        select_col.extend([x for x in range(th+1, temp_stock_df.shape[1])])

        temp_df = pd.merge(temp_stock_df.iloc[:, select_col], currency.loc[:, ['Date', 'Close']], on = 'Date', how = 'inner')
        stock_df = temp_stock_df.iloc[:, :(th+1)]

        def exchange(x):
            return x * temp_df['Close']

        temp_df.iloc[:, 1:-1] = temp_df.iloc[:, 1:-1].apply(exchange, axis = 0)
        stock_df = pd.merge(stock_df, temp_df, on = 'Date', how = 'inner')
        del stock_df['Close']

        stock_df.to_excel(data_path + "stock_df.xlsx", index = False)

    else:
        data = data.iloc[1:, :]

        stock_list = data['종목명']
        symbol_list = list(df_infos.loc[df_infos.Name.isin(stock_list)].Symbol)

        start_idx = 0

        for idx, symbol in enumerate(tqdm(symbol_list[start_idx:])):
            try:
                df = fdr.DataReader(symbol, start = min_date)

            except ValueError:
                start_idx += 1
                continue

            if df.shape[0] == 0:
                start_idx += 1
                continue

            name = change_name(symbol, df_infos, True)

            if start_idx == 0 :
                stock_df = make_df(name, df)
            else:
                temp_df = make_df(name, df)
                stock_df = pd.merge(stock_df, temp_df, on ='Date', how = 'outer')

            start_idx += 1

        stock_df = stock_df.sort_values("Date")
        stock_df.index = [x for x in range(stock_df.shape[0])]

        stock_df.to_excel(data_path + "stock_df.xlsx", index = False)
    return stock_df

* about portfolio

In [3]:
def get_perf_stats(returns, factor_returns=None, positions=None,
                    transactions=None, turnover_denom='AGB',
                    live_start_date=None, bootstrap=False,
                    header_rows=None):
    if bootstrap:
        perf_func = pf.timeseries.perf_stats_bootstrap
    else:
        perf_func = pf.timeseries.perf_stats
    STAT_FUNCS_PCT = [
        'Annual return',
        'Cumulative returns',
        'Annual volatility',
        'Max drawdown',
        'Daily value at risk',
        'Daily turnover'
    ]

    perf_stats_all = perf_func(
        returns,
        factor_returns=factor_returns,
        positions=positions,
        transactions=transactions,
        turnover_denom=turnover_denom)

    date_rows = OrderedDict()
    if len(returns.index) > 0:
        date_rows['Start date'] = returns.index[0].strftime('%Y-%m-%d')
        date_rows['End date'] = returns.index[-1].strftime('%Y-%m-%d')

    if live_start_date is not None:
        live_start_date = ep.utils.get_utc_timestamp(live_start_date)
        returns_is = returns[returns.index < live_start_date]
        returns_oos = returns[returns.index >= live_start_date]

        positions_is = None
        positions_oos = None
        transactions_is = None
        transactions_oos = None

        if positions is not None:
            positions_is = positions[positions.index < live_start_date]
            positions_oos = positions[positions.index >= live_start_date]
            if transactions is not None:
                transactions_is = transactions[(transactions.index <
                                                live_start_date)]
                transactions_oos = transactions[(transactions.index >
                                                 live_start_date)]

        perf_stats_is = perf_func(
            returns_is,
            factor_returns=factor_returns,
            positions=positions_is,
            transactions=transactions_is,
            turnover_denom=turnover_denom)

        perf_stats_oos = perf_func(
            returns_oos,
            factor_returns=factor_returns,
            positions=positions_oos,
            transactions=transactions_oos,
            turnover_denom=turnover_denom)
        if len(returns.index) > 0:
            date_rows['In-sample months'] = int(len(returns_is) /
                                                APPROX_BDAYS_PER_MONTH)
            date_rows['Out-of-sample months'] = int(len(returns_oos) /
                                                    APPROX_BDAYS_PER_MONTH)

        perf_stats = pd.concat(OrderedDict([
            ('In-sample', perf_stats_is),
            ('Out-of-sample', perf_stats_oos),
            ('All', perf_stats_all),
        ]), axis=1)
    else:
        if len(returns.index) > 0:
            date_rows['Total months'] = int(len(returns) /
                                            APPROX_BDAYS_PER_MONTH)
        perf_stats = pd.DataFrame(perf_stats_all, columns=['Backtest'])

    for column in perf_stats.columns:
        for stat, value in perf_stats[column].iteritems():
            if stat in STAT_FUNCS_PCT:
                perf_stats.loc[stat, column] = str(np.round(value * 100,
                                                            3)) + '%'
    if header_rows is None:
        header_rows = date_rows
    else:
        header_rows = OrderedDict(header_rows)
        header_rows.update(date_rows)
    return perf_stats

def get_price(df, date, name):
    while True :
        if np.isnan(df.loc[df.Date == date, name].values[0]):
            date -= pd.Timedelta(days = 1)
            
            while True:
                if date not in list(df.Date):
                    date -= pd.Timedelta(days = 1)
                else:
                    break
        else:
            break
    return df.loc[df.Date == date, name].values[0]

## change에 따른 자산 변화
def cal_changes(initial_value, df, col):
    change_values = [initial_value]
    
    cnt = 0
    for pct in df.loc[:, col]:
        if np.isnan(pct):
            pct = 0
        change_values.append(change_values[cnt] + change_values[cnt] * pct)
        cnt += 1
    return change_values

def get_return(port_input, opt = 'mean'):
    if opt == 'mean':
        output = mean_historical_return(port_input)
    elif opt == 'ema':
        output = ema_historical_return(port_input)
    elif opt == 'capm':
        output = capm_return(port_input)
    elif opt == "price":
        output = returns_from_prices(port_input)
    return output

def get_cov(port_input, opt = 'CovarianceShrinkage-01'):
    if opt == 'CovarianceShrinkage-01':
        S = CovarianceShrinkage(port_input).ledoit_wolf()
    elif opt == 'CovarianceShrinkage-02':
        S = CovarianceShrinkage(port_input).shrunk_covariance()
    elif opt == 'CovarianceShrinkage-03':
        S = CovarianceShrinkage(port_input).oracle_approximating()
    elif opt == "exp_cov":
        S = pypfopt.risk_models.exp_cov(port_input)
    elif opt == "semicovariance":
        S = pypfopt.risk_models.semicovariance(port_input)
    return S

def mvo(ef, method = "max_sharpe"):
    if method == 'max_sharpe':
        weights = ef.max_sharpe()
    elif method == "min_volatility":
        weights = ef.min_volatility()
    elif method == "max_quadratic_utility":
        weights = ef.max_quadratic_utility()
    return weights

def obj_functions(ef, label):
    if label == "Max Sharpe":
        return ef.max_sharpe()
    elif label == "Min Volatility":
        return ef.min_volatility()

def plot_pie(weights_df, save_img_path, inches = (12, 8)):
    fig, ax = plt.subplots(figsize=(32, 32), subplot_kw=dict(aspect="equal"))
    data = list(weights_df.allocations)
    ingredients = list(weights_df.index)

    def func(pct, allvals):
        absolute = int(pct/100.*np.sum(allvals))
        return "{:.1f}%".format(pct)

    wedges, texts, autotexts = ax.pie(data, autopct=lambda pct: func(pct, data),
                                      textprops=dict(color="w"))
    ax.legend(wedges, ingredients,
              title="Stocks",
              loc="center left",
              bbox_to_anchor=(1, 0, 0.5, 1))
    plt.setp(autotexts, size=13, weight="bold")
    ax.set_title("Portfolio Allocations", size = 15)
    plt.gcf().set_size_inches(inches)
    plt.tight_layout()
    plt.savefig(save_img_path + 'Portfolio Allocations.png', facecolor='#eeeeee', edgecolor='black')
    plt.close()

def plot_abount_returns(returns, positions, bench_rets, save_img_path, bench_name, inches = (12, 8)):
    ## rolling returns
    pf.plot_rolling_returns(returns=returns.loc[returns.index.isin(bench_rets.index)], 
                        factor_returns=bench_rets.pct_change(),
                        live_start_date=pd.Timestamp('2022-07-01'),
                        cone_std = [1.0, 1.5, 2.0])
    plt.gcf().set_size_inches(inches)
    plt.title("Portfolio Growth", size = 15)
    plt.savefig(save_img_path + 'Portfolio Growth.png',facecolor='#eeeeee')
    plt.close()
    
    ## RETURNS
    pf.plot_returns(returns)
    plt.gcf().set_size_inches(inches)
    plt.title("Returns", size = 15)
    plt.savefig(save_img_path + 'Returns.png', facecolor='#eeeeee')
    plt.close()
    
    ## return and groth
    fig, ax = plt.subplots(nrows = 2, ncols=1, figsize = (12, 16))
    axes = ax.flatten()
    pf.plot_rolling_returns(returns=returns.loc[returns.index.isin(bench_rets.index)], 
                            factor_returns=bench_rets.pct_change(),
                            live_start_date=pd.Timestamp('2022-07-01'),
                            cone_std = [1.0, 1.5, 2.0],
                           ax = axes[0],
                           title = "Portfolio Growth")
    pf.plot_returns(returns, ax = axes[1])
    plt.tight_layout()
    plt.title("Return")
    plt.savefig(save_img_path + 'Growth & Return.png', facecolor='#eeeeee')
    plt.close()

    ## annual returns
    pf.plot_annual_returns(returns)
    plt.gcf().set_size_inches(inches)
    plt.title("Annual Returns", size = 15)
    plt.legend(["Return"])
    plt.tight_layout()
    plt.savefig(save_img_path + 'Annual Returns.png', facecolor='#eeeeee')
    plt.close()
    
    ## monthly returns
    pf.plot_monthly_returns_heatmap(returns)
    plt.gcf().set_size_inches(inches)
    plt.title("Monthly Returns (%)", size = 15)
    plt.savefig(save_img_path + 'Monthly Returns (%).png', facecolor='#eeeeee')
    plt.close()
    
    pf.plot_monthly_returns_timeseries(returns)
    plt.gcf().set_size_inches(inches)
    plt.title("Monthly Returns Timeseries", size = 15)
    plt.savefig(save_img_path + 'Monthly Returns Timeseries.png', facecolor='#eeeeee')
    plt.close()
    
    pf.plot_monthly_returns_dist(returns)
    plt.gcf().set_size_inches(inches)
    plt.title("Distribution of Monthly Returns", size = 15)
    plt.legend(["Return"])
    plt.savefig(save_img_path + 'Distribution of Monthly Returns.png', facecolor='#eeeeee')
    plt.close()
    
    ## return and groth
    fig, ax = plt.subplots(nrows = 2, ncols=2, figsize = (20, 12))
    axes = ax.flatten()

    pf.plot_annual_returns(returns, ax = axes[0])
    plt.title("Annual Returns")
    plt.legend(["Return"])

    pf.plot_monthly_returns_heatmap(returns, ax = axes[2])
    plt.title("Monthly Returns (%)")

    pf.plot_monthly_returns_dist(returns, ax = axes[1])
    plt.title("Distribution of Monthly Returns")

    pf.plot_monthly_returns_timeseries(returns, ax = axes[3])
    plt.title("Monthly Returns Timeseries")
    plt.tight_layout()
    plt.savefig(save_img_path + 'Monthly & Annualy Returns.png', facecolor='#eeeeee')
    plt.close()
    
    pf.plot_return_quantiles(returns)
    plt.gcf().set_size_inches(inches)
    plt.title("Returns Quantiles per Periods", size = 15)
    plt.savefig(save_img_path + 'Returns Quantiles per Periods.png', facecolor='#eeeeee')
    plt.close()
    
    pf.plotting.plot_perf_stats(returns, None)
    plt.gcf().set_size_inches(inches)
    plt.savefig(save_img_path + 'Portfolio Summary.png', facecolor='#eeeeee')
    plt.close()
    
    pf.plotting.plot_rolling_beta(returns, bench_rets.pct_change())
    plt.gcf().set_size_inches(inches)
    plt.ylim([-1, 1.5])
    plt.title("Rolling Portfolio Beta to {}".format(bench_name), size = 15)
    plt.savefig(save_img_path + "Rolling Portfolio Beta to {}.png".format(bench_name),facecolor='#eeeeee')
    plt.close()
    
    pf.plot_rolling_sharpe(returns)
    plt.gcf().set_size_inches(inches)
    plt.title("Rolling Sharpe Ratio (6-month)", size = 15)
    plt.savefig(save_img_path + 'Rolling Sharpe Ratio (6-month).png', facecolor='#eeeeee')
    plt.close()
    
    pf.plot_drawdown_periods(returns,top = 5)
    plt.gcf().set_size_inches(inches)
    plt.title("Top 5 Drawdown Periods", size = 15)
    plt.savefig(save_img_path + 'Top 5 Drawdown Periods.png', facecolor='#eeeeee')
    plt.close()
    
    pf.plot_drawdown_underwater(returns)
    plt.gcf().set_size_inches(inches)
    plt.title("Underwater Plot", size = 15)
    plt.savefig(save_img_path + 'Underwater Plot.png', facecolor='#eeeeee')
    plt.close()
    
    ## total 4
    fig, ax = plt.subplots(nrows = 2, ncols=2, figsize = (20, 12))
    axes = ax.flatten()

    pf.plot_drawdown_periods(returns, top = 5, ax = axes[0])
    plt.title("Top 5 Drawdown Periods", size = 15)

    pf.plotting.plot_rolling_beta(returns, bench_rets.pct_change(), ax = axes[1], ylim = [-1, 1.5])
    plt.title("Rolling Portfolio Beta to {}".format(bench_name), size = 15)

    pf.plot_drawdown_underwater(returns, ax = axes[2])
    plt.title("Underwater Plot", size = 15)

    pf.plot_rolling_sharpe(returns, ax = axes[3])
    plt.title("Rolling Sharpe Ratio (6-month)", size = 15)
    plt.tight_layout()
    plt.savefig(save_img_path + 'Factor & Drawdown.png', facecolor='#eeeeee')
    plt.close()
    
    pf.plotting.show_and_plot_top_positions(returns, positions, show_and_plot=0)
    plt.gcf().set_size_inches(inches)
    plt.title("Portfolio Allocation over Time, Only Top 10 Holdings", size = 15)
    plt.savefig(save_img_path + 'Portfolio Allocation over Time, Only Top 10 Holdings.png', facecolor='#eeeeee')
    plt.close()

* about tkinter

In [4]:
# 데이터 파일 추가
def add_file():
    files = filedialog.askopenfilenames(title="데이터 선택", \
        filetypes=(("모든 파일", "*.*"),
                ("엑셀 파일", "*.xlsx"),
                ("csv 파일", "*.csv")),\
        initialdir="./datasets/")
    
    for idx, file in enumerate(files):
        list_file.insert(parent="", index = "end", iid = idx,text = file, value = (END, file))
        
# 선택 삭제
def del_file():
    selected_item = list_file.selection()[0]
    list_file.delete(selected_item)
    
# 저장 경로 (폴더)
def browse_dest_path():
    folder_selected = filedialog.askdirectory()
    if folder_selected is None: # 사용자가 취소를 누를 때
        return
    txt_dest_path.delete(0, END)
    txt_dest_path.insert(0, folder_selected)

def read_xlsx(name):
    instance = xw.App(visible=False)
    xlsx_data = xw.Book(name).sheets[0]
    df = xlsx_data.range('A1').options(pd.DataFrame, index = False, expand = 'table').value
    instance.quit()
    instance.kill()
    return df

# load data
def main_functions():
    global pf_year
    complete = True
    
    ## 저장경로 에러
    if balance_space.get() == "":
        msgbox.showerror("에러", "초기 금액을 설정하세요.")
        complete = False
        
    else:
        ## 01. data-include part
        data_path = os.getcwd() + "\\datasets\\"
        currency, _ = get_currency(data_path)
        df_infos = load_data(data_path)
        year = pf_year.get()
        start_date = str(year) + "-01-01"
        
        ## load files
        '''
        datas = []
        for parent in list_file.get_children():
            datas.append(read_data(list_file.item(parent)["values"][1]))

        if len(datas) == 1:
            data = datas[0]
        '''
        
        data = read_data("./datasets/" + data_space.get())
        
        stock_df = get_include_data(data, df_infos, currency, data_path, start_date)
        port_input = stock_df.iloc[:, 1:]
        #print(stock_df.shape) (8433, 25)
        #print(port_input.shape) (8433, 24)
        
        rebalancing_days = {"Monthly":30, "Quarterly": 90, "Semi-annually": 180, "Annually":365}
        DAYS = pf_rebal.get()
        #print(rebalancing_days[pf_rebal.get()]) 30
        
        algo_name = pf_algo.get()
        obj_name = pf_obj.get()
        bench_name = pf_bench.get()
        initial_balance = int(balance_space.get())
        currency.Date = pd.to_datetime(currency.Date, utc = True)
        
        #print(algo_name, obj_name, bench_name, initial_balance, type(initial_balance)) [MVO] Mean-Variance Optimization  Max Sharpe KOSPI 1000000 <class 'str'>
        
        ## 02.portfolio part
        if bench_name == "KOSPI":
            bench_ret = fdr.DataReader('KS11', start_date)['Close']
            bench_ret.index = pd.to_datetime(bench_ret.index, utc = True)
            exchange = currency.loc[currency.Date.isin(bench_ret.index)]['Close']
            exchange.index = bench_ret.index
            bench_ret *= exchange
            bench_ret.name = 'KOSPI'
        
        elif bench_name == "S&P500":
            bench_ret = fdr.DataReader('US500', start_date)['Close']
            bench_ret.index = pd.to_datetime(bench_ret.index, utc = True)
            exchange = currency.loc[currency.Date.isin(bench_ret.index)]['Close']
            exchange.index = bench_ret.index
            bench_ret *= exchange
            bench_ret.name = 'S&P500'

        start_date = pd.to_datetime(start_date, utc = True)
        
        ## initial settings
        total_amount = initial_balance
        end_date = start_date + pd.Timedelta(days = rebalancing_days[DAYS])
        re_date = start_date
        utc_time = [pd.to_datetime(x, utc = True) for x in stock_df.Date]
        stock_df.Date = utc_time

        # make pf_positions
        pf_positions = pd.DataFrame({x:[0] for x in stock_df.columns[1:]})
        pf_positions['cash'] = initial_balance
        pf_positions.index = [end_date - pd.Timedelta(days=1)]

        # make pf_transactions
        pf_transactions = pd.DataFrame({x:[0] for x in ['amount', 'order_id', 'price', 'sid', 'symbol', 'txn_dollars']})
        pf_transactions.index = [end_date - pd.Timedelta(days=1)]

        loop_time_list = [start_date]
        weights_dict = {}

        total_cnt = 0
        
        while True:
            end_date = re_date + pd.Timedelta(days = rebalancing_days[DAYS])
            #print(end_date)

            ## end date 보정
            if not end_date >= utc_time[-1]:
                while True:
                    if end_date not in list(stock_df.Date):
                        end_date += pd.Timedelta(days = 1)
                    elif end_date in list(stock_df.Date): 
                        break
            else:
                real_df = stock_df.loc[(stock_df.Date >= re_date) & (stock_df.Date < end_date)]
                real_df_pct = real_df.iloc[:, 1:].pct_change().iloc[1:, :]

                ## 0이 아닌 컬럼 선택
                not_zero_idx = np.where(pf_positions.loc[re_date,:] != 0)[0]
                select_col = list(pf_positions.columns[not_zero_idx])
                select_col.remove("cash")

                ## 종목 변화 계산
                temp_df = pd.DataFrame()
                for col in select_col:
                    initial_value = pf_positions.loc[re_date, col]
                    temp_df[col] = cal_changes(initial_value, real_df_pct, col)

                # temp_df date 중 추가할 부분만 선택
                temp_df.index = real_df.Date
                temp_df = temp_df.loc[temp_df.index > np.max(pf_positions.index)]

                # pr positions 갱신
                pf_positions = pf_positions.reset_index()
                pf_positions = pf_positions.rename(columns={"index":"Date"})
                temp_df = temp_df.reset_index()

                on_list = ['Date']
                on_list.extend(select_col)

                pf_positions = pd.merge(pf_positions, temp_df, on = on_list, how = "outer")

                pf_positions.index = pf_positions.Date
                pf_positions = pf_positions.iloc[:, 1:]

                # cash 마지막 값 복붙
                pf_positions.loc[pf_positions.index > re_date, 'cash'] = pf_positions.loc[re_date, 'cash']
                pf_positions.replace(np.NaN, 0, inplace = True)
                
                # get final weights
                df = stock_df.loc[(stock_df.Date >= start_date) & (stock_df.Date < end_date)]
                df = df.loc[:, ~df.columns.isin(['Date'])]
                df = df.iloc[:, np.where(df.isna().sum() / df.shape[0] != 1)[0]]
                if algo_name == "[MVO] Mean-Variance Optimization":
                    ef = EfficientFrontier(mean_return, cs_01)
                    weights = obj_functions(ef, obj_name)

                elif algo_name == "[HRP] Hierarchical Risk Parity":
                    rp = pypfopt.hierarchical_portfolio.HRPOpt(price_return, cs_01)
                    weights = rp.optimize()

                elif algo_name == "[CLA] The Critical Line Algorithm":
                    cla = CLA(mean_return, cs_01, weight_bounds=(0, 1))
                    weights = obj_functions(cla, obj_name)
                final_weights = weights
                break

            loop_time_list.append(end_date)
            df = stock_df.loc[(stock_df.Date >= start_date) & (stock_df.Date < end_date)]
            df = df.loc[:, ~df.columns.isin(['Date'])]
            df = df.iloc[:, np.where(df.isna().sum() / df.shape[0] != 1)[0]]
            
            # 최적화 포트 계산         
            # get return
            #capm_return = get_return(df, 'capm')
            price_return = get_return(df, 'price')
            mean_return = get_return(df, 'mean')
            #ema_return = get_return(df, 'ema')
        
            # get cov
            cs_01 = get_cov(df, opt = 'CovarianceShrinkage-01')
            #cs_02 = get_cov(df, opt = 'CovarianceShrinkage-02')
            #cs_03 = get_cov(df, opt = 'CovarianceShrinkage-03')
            #S_exp = get_cov(df, opt = 'exp_cov')
            #S_semi = get_cov(df, opt = 'semicovariance')
            
            try:
                if algo_name == "[MVO] Mean-Variance Optimization":
                    ef = EfficientFrontier(mean_return, cs_01)
                    weights = obj_functions(ef, obj_name)

                elif algo_name == "[HRP] Hierarchical Risk Parity":
                    rp = pypfopt.hierarchical_portfolio.HRPOpt(price_return, cs_01)
                    weights = rp.optimize()

                elif algo_name == "[CLA] The Critical Line Algorithm":
                    cla = CLA(mean_return, cs_01, weight_bounds=(0, 1))
                    weights = obj_functions(cla, obj_name)
            except:
                msgbox.showerror("최적화 에러", "다른 Optimizer 혹은 Objective를 선택하세요.")
                complete = False
                break
                
            weights_dict[end_date] = weights

            ### 모든 포지션 청산
            ## position 갱신
            if total_cnt > 0:
                real_df = stock_df.loc[(stock_df.Date >= re_date) & (stock_df.Date < end_date)]
                real_df_pct = real_df.iloc[:, 1:].pct_change().iloc[1:, :]

                ## 0이 아닌 컬럼 선택
                not_zero_idx = np.where(pf_positions.loc[re_date,:] != 0)[0]
                select_col = list(pf_positions.columns[not_zero_idx])
                select_col.remove("cash")

                ## 종목 변화 계산
                temp_df = pd.DataFrame()
                for col in select_col:
                    initial_value = pf_positions.loc[re_date, col]
                    temp_df[col] = cal_changes(initial_value, real_df_pct, col)

                # temp_df date 중 추가할 부분만 선택
                temp_df.index = real_df.Date
                temp_df = temp_df.loc[temp_df.index > np.max(pf_positions.index)]

                # pr positions 갱신
                pf_positions = pf_positions.reset_index()
                pf_positions = pf_positions.rename(columns={"index":"Date"})
                temp_df = temp_df.reset_index()

                on_list = ['Date']
                on_list.extend(select_col)

                pf_positions = pd.merge(pf_positions, temp_df, on = on_list, how = "outer")

                pf_positions.index = pf_positions.Date
                pf_positions = pf_positions.iloc[:, 1:]

                # cash 마지막 값 복붙
                pf_positions.loc[pf_positions.index > re_date, 'cash'] = pf_positions.loc[re_date, 'cash']
                pf_positions.replace(np.NaN, 0, inplace = True)

                append_positions = deepcopy(pf_positions.tail(1))
                append_positions.index = [end_date]

                append_transactions = deepcopy(pf_transactions.tail(1))
                append_transactions.index = [end_date]

                ## 마지막 total money 계산
                total_amount = pf_positions.iloc[:, not_zero_idx].tail(1).sum(axis = 1).values[0]

                ## 청산 코드
                ratio = -1
                for name in select_col:
                    get_values = get_price(stock_df, end_date, name)

                    # append_transactions
                    tot_amts = pf_transactions.loc[pf_transactions.sid == name].amount.sum()

                    # transactions
                    append_transactions['amount'] = int(tot_amts * ratio)
                    append_transactions['order_id'] = str(total_cnt)
                    append_transactions['price'] = get_values
                    append_transactions['sid'] = name
                    append_transactions['symbol'] = name
                    append_transactions['txn_dollars'] = -(int(tot_amts * ratio) * get_values)

                    pf_transactions = pd.concat([pf_transactions, append_transactions])

                    # positions
                    append_positions.loc[:, 'cash'] -= append_positions.loc[end_date, name] * ratio
                    append_positions.loc[:, name] += append_positions.loc[end_date, name] * ratio

                    total_cnt += 1

                if end_date >= utc_time[-1]:
                    break

            else:
                append_positions = deepcopy(pf_positions.tail(1))
                append_positions.index = [end_date]

                append_transactions = deepcopy(pf_transactions.tail(1))
                append_transactions.index = [end_date]

            ## 구매 코드
            buy_dict = {x:np.round(y, 4) for x,y in zip(weights.keys(), weights.values())}

            # 구매할 종목 및 금액
            buy_list = []
            buy_amt = []

            for st_name, percent in weights.items():
                percent = np.round(percent, 4)
                if percent > 0:
                    buy_amt.append(np.round(total_amount * percent, 3))
                    buy_list.append(st_name)

            buy_cnt = 0
            for name, amt in zip(buy_list, buy_amt):
                get_values = get_price(stock_df, end_date, name)
                amount = int(buy_amt[buy_cnt] / get_values)
                total_amt = get_values * amount
                append_positions[name] = total_amt
                append_positions['cash'] -= total_amt

                # transactions
                append_transactions['amount'] = amount
                append_transactions['order_id'] = str(total_cnt)
                append_transactions['price'] = get_values
                append_transactions['sid'] = name
                append_transactions['symbol'] = name
                append_transactions['txn_dollars'] = -total_amt

                pf_transactions = pd.concat([pf_transactions, append_transactions])

                buy_cnt += 1
                total_cnt += 1

            pf_positions = pd.concat([pf_positions, append_positions])

            # 날짜 업데이트
            re_date = end_date
            before_date = pf_positions.tail(1).index[0]
            ## end of while
            
        pf_returns = pf_positions.sum(axis = 1).pct_change().iloc[1:]
        
        ## benchmark
        bench_ret = bench_ret.loc[bench_ret.index.isin(pf_returns.index)]
        
        save_img_path = data_path + "\\imgs\\"

        if not os.path.isdir(save_img_path):
            os.mkdir(save_img_path)
        
        ## save fig
        plot_abount_returns(pf_returns, pf_positions, bench_ret, save_img_path, bench_name, inches = (12, 8))
        
        ## pie chart
        weights_df = pd.DataFrame.from_dict(final_weights, orient='index', columns=['allocations'])
        weights_df = weights_df.loc[weights_df.allocations != 0]
        plot_pie(weights_df, save_img_path)
        
        ## save markdown
        mdFile = MdUtils(file_name='Portfolio Report',title='HUG-Portfolio Reports')
        mdFile.create_md_file()
        mdFile.new_line("<br>")

        mdFile.new_header(level=1, title='')
        start_str = "{year}.{month}.{day}".format(year = start_date.year, month = start_date.month, day = start_date.day)
        now_str = "{year}.{month}.{day}".format(year = datetime.now().year, month = datetime.now().month, day = datetime.now().day)
        mdFile.new_header(level=2, title='01. Portfolio Allocations ({}~{})'.format(start_str, now_str))
        mdFile.new_line("<br>")

        mdFile.new_line("최적화된 포트폴리오의 샤프 비율, 시장 베타, 최대 손실폭은 다음과 같다.",bold_italics_code='bold')
        mdFile.new_line("<br>")
        mdFile.new_line("1. The Sharpe Ratio of the backtest is (샤프 비율): {}".format(np.round(empyrical.sharpe_ratio(pf_returns), 4)))
        mdFile.new_line("2. The market beta of the backtest is (시장 베타): {}".format(np.round(empyrical.beta(pf_returns, bench_ret.pct_change()),4)))
        mdFile.new_line("3. The maxmimum drawdown of the backtest is (최대 손실폭): {}".format(np.round(empyrical.max_drawdown(pf_returns), 4)))

        mdFile.new_line("<br>")
        mdFile.new_line("현재 시점으로 보유 종목의 최적 배분은 다음과 같다.",bold_italics_code='bold')
        mdFile.new_header(level=2, title='')

        weights_df = pd.DataFrame.from_dict(weights, orient='index', columns=['Allocations'])
        weights_df = weights_df.loc[weights_df.Allocations != 0]
        weights_df['Balance'] = np.array(pf_positions.tail(1).loc[:, weights_df.index])[0]
        append_weights = pd.DataFrame({"Allocations":[0.0], "Balance": [np.array(pf_positions.tail(1)['cash'])[0]]}, index = ['현금'])
        weights_df = pd.concat([weights_df, append_weights])
        weights_df['Allocations'] = [str(np.round(x,4)) for x in weights_df['Allocations']]
        weights_df['Balance'] = [str(np.int(x)) for x in weights_df['Balance']]

        mdFile.write(weights_df.to_markdown() + "\n")

        mdFile.new_header(level=3, title='')
        mdFile.new_header(level=4, title='[표 1] 최적 포트폴리오 배분')
        mdFile.new_line("<br>")

        mdFile.new_line(mdFile.new_inline_image(text='Portfolio Allocations', path='./datasets/imgs/Portfolio Allocations.png'))

        mdFile.new_header(level=4, title='[그림 1] 최적 포트폴리오 배분 파이 차트')
        mdFile.new_line("<br>")

        mdFile.new_header(level=2, title='02. 포트폴리오 성과 통계량 요약')

        mdFile.new_line("<br>")
        #mdFile.new_line("포트폴리오 전체 기간에 대한 측정 항목 및 통계량은 다음과 같다.", bold_italics_code='bold')
        mdFile.new_line("포트폴리오 전체 기간에 대한 측정 항목 및 통계량은 다음과 같다.")

        mdFile.new_header(level=2, title='')
        port_summary = get_perf_stats(pf_returns)
        port_summary.columns = ['성과 통계량']
        temp_idx = ['연간 수익률', '누적 수익률', '연간 변동성', '샤프 비율', '칼마 비율', '안정성', '최대 손실 폭', '오메가 비율', '솔티노 비율', '왜도', '첨도', '테일 비율', '일별 VaR']
        port_summary.index = [x+" ({})".format(y) for x,y in zip(temp_idx, port_summary.index)]
        mdFile.write(port_summary.to_markdown() + "\n")

        mdFile.new_header(level=3, title='')
        mdFile.new_header(level=4, title='[표 2] 포트폴리오 성과 통계표')
        mdFile.new_line("<br>")

        mdFile.new_line(mdFile.new_inline_image(text='Portfolio Summary', path='./datasets/imgs/Portfolio Summary.png'))
        mdFile.new_header(level=4, title='[그림 2] 포트폴리오 성과 통계 도표')

        mdFile.new_line("<br>")
        mdFile.new_header(level=2, title='03. 포트폴리오 백테스팅 결과')
        mdFile.new_header(level=3, title='03.1 포트폴리오 Growth & Returns')
        mdFile.new_line("<br>")
        mdFile.new_line("벤치마크 대비 포트폴리오의 기간 내 성과는 다음과 같다.")
        mdFile.new_header(level=2, title='')
        mdFile.new_header(level=3, title='')

        mdFile.new_line(mdFile.new_inline_image(text='Growth & Return', path='./datasets/imgs/Growth & Return.png'))
        mdFile.new_header(level=4, title='[그림 3] 포트폴리오의 기간 내 Growth & Returns')

        mdFile.new_line("<br>")
        mdFile.new_header(level=3, title='03.2 포트폴리오 월/연도별 Returns')
        mdFile.new_line("<br>")
        mdFile.new_line("포트폴리오의 연도별 Returns는 다음과 같다.")

        mdFile.new_header(level=3, title='')
        pf_year = pd.DataFrame(pf.timeseries.aggregate_returns(pf_returns, 'yearly'))
        pf_year.index.name = ''
        pf_year.columns = ['Return']
        mdFile.write(pf_year.to_markdown() + "\n")
        mdFile.new_header(level=4, title='[표 3] 연도별 Returns')

        mdFile.new_line("<br>")
        mdFile.new_line("포트폴리오의 연도별, 월별 Returns를 도식화하면 그림 4와 같으며 좌측 상단부터 시계방향으로 설명하면 다음과 같다.   ", bold_italics_code='bold')
        mdFile.new_line("<br>")
        mdFile.new_line("1. 연도별 Returns")
        mdFile.new_line("2. 월별 Returns의 분포")
        mdFile.new_line("3. 연도-월별 Returns의 막대 그래프")
        mdFile.new_line("4. 연도-월별 Returns의 히트맵")
        mdFile.new_line(" ")

        mdFile.new_line(mdFile.new_inline_image(text='Annual Returns', path='./datasets/imgs/Monthly & Annualy Returns.png'))
        mdFile.new_header(level=4, title='[그림 4] 월 & 연도별 Returns 도표')
        mdFile.new_line("<br>")



        mdFile.new_line("<br>")
        mdFile.new_header(level=3, title='03.3 포트폴리오 하락 기간과 팩터 노출')
        mdFile.new_line("<br>")
        mdFile.new_line("해당 포트의 주요 하락 기간은 다음과 같다.")

        mdFile.new_header(level=3, title='')
        port_ddw = pf.timeseries.gen_drawdown_table(pf_returns, top = 5)
        mdFile.write(port_ddw.to_markdown() + "\n")
        mdFile.new_header(level=4, title='[표 4] Top 5 하락 기간')

        mdFile.new_line("<br>")
        mdFile.new_line("포트폴리오의 연도별, 월별 Returns를 도식화하면 그림 4와 같으며 좌측 상단부터 시계방향으로 설명하면 다음과 같다.   ", bold_italics_code='bold')
        mdFile.new_line("<br>")
        mdFile.new_line("1. Top 5 손실 기간 ")
        mdFile.new_line("2. 롤링 포트폴리오 베타")
        mdFile.new_line("3. Underwater Plot")
        mdFile.new_line("4. 롤링 샤프 비율 (6개월)")
        mdFile.new_line(" ")

        mdFile.new_line(mdFile.new_inline_image(text='Factor & Drawdown', path='./datasets/imgs/Factor & Drawdown.png'))
        mdFile.new_header(level=4, title='[그림 5] 시간 경과에 따른 하락 기간 및 팩터 노출 그래프')
        mdFile.new_line("<br>")
        
        mdFile.create_md_file()
k
        markdown.markdownFromFile(
            input='Portfolio Report.md',
            output='Portfolio Report.html',
            extensions=['markdown.extensions.tables'],
            encoding='utf8'
        )
        if complete :
            msgbox.showinfo("알림", "작업이 완료되었습니다.")

* layout

In [7]:
rebalancing_days = {"Monthly":30, "Quarterly": 90, "Semi-annually": 180, "Annually":365}

root = Tk()
root.title("HUG-Portfolio Analysis")
root.option_add("*tearOff", False)

# Create a style
style = ttk.Style(root)

root.tk.call("source", "./add/forest-light.tcl")
style.theme_use("forest-light")
root.update_idletasks()

# 파일 프레임 (파일 추가, 선택 삭제)
file_frame = ttk.LabelFrame(root, text = "WINK 1171 or WINK 6002")
file_frame.pack(fill="x", padx=5, pady=5) # 간격 띄우기

btn_frame = Frame(file_frame)
btn_frame.pack(side = "right")

data_space = ttk.Entry(file_frame, width = 85)
data_space.pack()

# 옵션 프레임
frame_option = ttk.Frame(root)
frame_option.pack(padx=5, pady=5, ipady=5)

notebook = ttk.Notebook(frame_option)
pf_options = ttk.Frame(notebook)

pf_left = ttk.Frame(pf_options)
pf_right = ttk.Frame(pf_options)

pf_left.pack(side = "left")
pf_right.pack(side = "right")

# 1. rebalancing settings
pf_cate_label = Label(pf_left, text="Rebalancing", width=15)
pf_cate_label.grid(row = 0, column=0, padx=2, pady=5)

opt_rebal_cate = list(rebalancing_days.keys())
pf_rebal = ttk.Combobox(pf_left, state="readonly", values=opt_rebal_cate, width=20)
pf_rebal.current(0)
pf_rebal.grid(row = 0, column=1, padx=2, pady=5)

# 2. optimization methods 선택
pf_optim_label = Label(pf_left, text="Optimizers", width=15)
pf_optim_label.grid(row = 1, column=0, padx=2, pady=5)

opt_pf_algo = ['[MVO] Mean-Variance Optimization', '[HRP] Hierarchical Risk Parity', '[CLA] The Critical Line Algorithm']
pf_algo = ttk.Combobox(pf_left, state="readonly", values=opt_pf_algo, width=20)
pf_algo.current(0)
pf_algo.grid(row = 1, column=1, padx=2, pady=5)

# 3. Objectvie function 선택
pf_obj_label = Label(pf_left, text="Objective", width=15)
pf_obj_label.grid(row = 2, column=0, padx=2, pady=5)

obj_label = ['Max Sharpe', 'Min Volatility']
pf_obj = ttk.Combobox(pf_left, state="readonly", values=obj_label, width=20)
pf_obj.current(0)
pf_obj.grid(row = 2, column=1, padx=2, pady=5)

# 4. Start yaer
pf_year_label = Label(pf_right, text="Start Year", width=15)
pf_year_label.grid(row = 0, column=0, padx=2, pady=5)

opt_start_year = [x for x in range(1990, 2023)]
pf_year = ttk.Combobox(pf_right, state="readonly", values=opt_start_year, width=20)
pf_year.current(0)
pf_year.grid(row = 0, column=1, padx=2, pady=5)

# 5. benchmark
pf_bench_label = Label(pf_right, text="Benchmark", width=15)
pf_bench_label.grid(row=1, column=0, padx=5, pady=5)

opt_bench = ['KOSPI', 'S&P500']
pf_bench = ttk.Combobox(pf_right, state="readonly", values=opt_bench, width=20)
pf_bench.current(0)
pf_bench.grid(row = 1, column=1, padx=2, pady=5)

# 6. initial balance
pf_balance_label = Label(pf_right, text="Initial Balance", width=15)
pf_balance_label.grid(row=2, column=0, padx=2, pady=5)

balance_space = ttk.Entry(pf_right, width = 23)
balance_space.grid(row=2, column=1, padx=2, pady=5)

notebook.add(pf_options, text = "Portfolio Settings")
notebook.pack()

# 실행 프레임
frame_run = Frame(root)
frame_run.pack(fill="x", padx=5, pady=5)

btn_close = ttk.Button(frame_run, text="Close", width=12, command=root.quit)
btn_close.pack(side="right", padx=5, pady=5)

btn_start = ttk.Button(frame_run, text="Start", width=12, command = main_functions)
btn_start.pack(side="right", padx=5, pady=5)

root.resizable(True, True)
root.mainloop()